# Relaxed Search(제대로) 가미한 코드

아는 선에서 일단 건드려봤고 1차적인 결과를 올림<br>
알고리즘 돌리는 코드들은 동환이형이 쓴 그대로 썼고,<br>
알고리즘에 따른 셀 분리, relaxed search, best model 등을 구하기만 함. <br>
한번 쭉 봐주고 갖고 놀아보고 이것저것 만져보면 뭐가 뭔지 대충 느낌 올거임.<br>
아님 말구<br>
<strong>plot 은 아직 하나도 안 넣음(ex. 동환이형의 시간축 x 그래프). data visualization 에 필수적인 것이니 꼭 넣어야함</strong>
<br><br>

### 5월 23일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 2분... 많으면 5분?(데스크탑 기준...)</strong><br>
데이터 양에 따라 차이가 남.


In [25]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=10

In [26]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy.stats
import random
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [27]:
dataname = "bundang3years_small1.csv"

In [28]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
#print(df.head())

df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)
# df.drop(['마트최소거리'],axis=1,inplace=True)
# df.drop(['지하철도보거리'],axis=1,inplace=True)
# df.drop(['용적률'],axis=1,inplace=True)
# df.drop(['대형병원최소거리'],axis=1,inplace=True)
#df.drop(['층'],axis=1,inplace=True)
# df.drop(['건폐율'],axis=1,inplace=True)
#df.drop(['총세대수'],axis=1,inplace=True)
# df.drop(['규모별월세가격지수'],axis=1,inplace=True)
# df.drop(['규모별수익률'],axis=1,inplace=True)
# df.drop(['동별인구수'],axis=1,inplace=True)
#print(df.head())

#데이터 형태 실수로 변환
#면적 거래금액 분리버전
Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

# 단위면적당 거래금액 버전
# df.drop(['전용면적'],axis=1,inplace=True)
# df.drop(['거래금액'],axis=1,inplace=True)
# Y = df['거래금액2'].values
# Y = Y.astype('float')
# X = df.drop(labels=['거래금액2'],axis=1)
# X = X.astype('float')

In [29]:
test_size = [0.1,0.15,0.2,0.25,0.3]
ts_len = len(test_size)

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [30]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

# best_RF_RMSE_diff=100
# best_RF_RMSE_train=0
# best_RF_RMSE_test=0
# best_RF_MAPE_train=0
# best_RF_MAPE_test=0
RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_tsize_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_tsize_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for ts in test_size:
    for a in Tree_nums:
        for b in Var_nums:
            random.seed(a*10)
            #훈련셋, 시험셋 분류 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)
        
            #랜덤포레스트
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b)
            RF_model.fit(X_train, Y_train)
            
            #예측값 생성
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            RF_RMSE_train_list.append(RF_RMSE_train)
            RF_RMSE_test_list.append(RF_RMSE_test)
            RF_MAPE_train_list.append(RF_MAPE_train)
            RF_MAPE_test_list.append(RF_MAPE_test)
            
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_par_diff_list.append(RF_RMSE_diff)
            RF_par_tsize_list.append(ts)
            RF_par_trees_list.append(a)
            RF_par_var_list.append(b)
            
#             if best_RF_RMSE_diff>RF_RMSE_diff:
#                 best_RF_RMSE_diff = RF_RMSE_diff
#                 best_RF_RMSE_train = RF_RMSE_train
#                 best_RF_RMSE_test = RF_RMSE_test
#                 best_RF_MAPE_train = RF_MAPE_train
#                 best_RF_MAPE_test = RF_MAPE_test
#                 RF_C = count+1
#                 RF_TS = ts
#                 RF_A = a
#                 RF_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Tree_num: {} , Var_num: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(RF_C,RF_TS,RF_A,RF_B,best_RF_RMSE_train,best_RF_RMSE_test,best_RF_RMSE_diff,best_RF_MAPE_train,best_RF_MAPE_test))
                


for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_min_vars_tsize_list.append(RF_par_tsize_list.pop(index))
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))


----progress...----
 0.71% , case:1 , test_size: 0.1 , Tree_num: 5 , Var_num: 1 , difference(%) : 3.69
 ---  train RMSE : 5876 , test RMSE : 5667
---  train MAPE : 19.85 , test MAPE : 19.33
 1.43% , case:2 , test_size: 0.1 , Tree_num: 5 , Var_num: 3 , difference(%) : 4.25
 ---  train RMSE : 3239 , test RMSE : 3107
---  train MAPE : 11.21 , test MAPE : 10.65
 2.14% , case:3 , test_size: 0.1 , Tree_num: 5 , Var_num: 5 , difference(%) : 10.61
 ---  train RMSE : 1980 , test RMSE : 2215
---  train MAPE : 7.07 , test MAPE : 7.52
 2.86% , case:4 , test_size: 0.1 , Tree_num: 5 , Var_num: 7 , difference(%) : 11.50
 ---  train RMSE : 1432 , test RMSE : 1618
---  train MAPE : 4.85 , test MAPE : 5.31
 3.57% , case:5 , test_size: 0.1 , Tree_num: 10 , Var_num: 1 , difference(%) : 0.39
 ---  train RMSE : 5811 , test RMSE : 5834
---  train MAPE : 19.78 , test MAPE : 20.77
 4.29% , case:6 , test_size: 0.1 , Tree_num: 10 , Var_num: 3 , difference(%) : 3.91
 ---  train RMSE : 2999 , test RMSE : 3121
--- 

In [33]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("test_size : {} , Tree_num : {}, Var_num : {}".format(RF_min_vars_tsize_list[i],RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = RF_min_vars_tsize_list[i]
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        #랜덤포레스트
        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b, max_features=9)
        RF_Best_model.fit(X_train, Y_train)

            #예측값 생성
        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

            #훈련셋 RMSE & MAPE
        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

            #시험셋 RMSE & MAPE
        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        
        count+=1
        progress = round(count/(10*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))
    
#     RF_Best_RMSE_train_list.append(temp_RF_Best_RMSE_train_list)
#     RF_Best_RMSE_test_list.append(temp_RF_Best_RMSE_test_list)
#     RF_Best_MAPE_train_list.append(temp_RF_Best_MAPE_train_list)
#     RF_Best_MAPE_test_list.append(temp_RF_Best_MAPE_test_list)
#     RF_Best_RMSE_diff_list.append(temp_RF_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(RF_Best_C,RF_Best_TS,RF_Best_A,RF_Best_B,RF_Best_min))

PARAMETERS FOR BEST RF...
test_size : 0.25 , Tree_num : 500, Var_num : 1
test_size : 0.25 , Tree_num : 500, Var_num : 3
test_size : 0.25 , Tree_num : 500, Var_num : 5
test_size : 0.25 , Tree_num : 500, Var_num : 7
test_size : 0.25 , Tree_num : 750, Var_num : 1
...testing...
 2.00% , case:1 , test_size: 0.25 , Tree_num: 500 , Var_num: 1 , difference(%) : 7.26
 ---  train RMSE : 5783 , test RMSE : 6236
---  train MAPE : 19.64 , test MAPE : 21.09
 4.00% , case:2 , test_size: 0.25 , Tree_num: 500 , Var_num: 1 , difference(%) : 3.96
 ---  train RMSE : 5927 , test RMSE : 5701
---  train MAPE : 20.31 , test MAPE : 18.87
 6.00% , case:3 , test_size: 0.25 , Tree_num: 500 , Var_num: 1 , difference(%) : 5.97
 ---  train RMSE : 5967 , test RMSE : 5631
---  train MAPE : 20.29 , test MAPE : 19.84
 8.00% , case:4 , test_size: 0.25 , Tree_num: 500 , Var_num: 1 , difference(%) : 1.32
 ---  train RMSE : 5835 , test RMSE : 5913
---  train MAPE : 19.83 , test MAPE : 20.53
 10.00% , case:5 , test_size: 0.2

In [34]:
rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,5878.700000,3057.800000,1886.400000,1307.450000,5853.950000
RMSE test Mean,5877.900000,3103.550000,2012.750000,1653.350000,5977.300000
RMSE train Stdev,54.940968,42.476582,22.102941,29.186427,60.822262
RMSE test Stdev,218.041258,129.779226,72.173316,107.836578,249.296631
MAPE train Mean,20.060000,10.939500,6.642500,4.426000,19.981000
MAPE test Mean,19.944000,11.012000,6.901000,5.067500,20.128000
MAPE train Stdev,0.205524,0.123064,0.072655,0.061025,0.171840
MAPE test Stdev,0.684050,0.387061,0.198290,0.158804,0.573024
RMSE diff(%) Mean,3.900966,3.898815,6.155289,20.465035,3.763095
RMSE diff(%) Stdev,2.343853,2.285417,4.053367,7.061390,3.673992


### 2. Support Vector Machine parameters

In [18]:
gamma_nums = [0.00015,0.001,0.01,0.1]
C_values= [500,1000,5000,10000,50000]
x = len(gamma_nums)
y = len(C_values)
count=0
# Random Forest Relaxed Search
count=0

# best_SVR_RMSE_diff=100
# best_SVR_RMSE_train=0
# best_SVR_RMSE_test=0
# best_SVR_MAPE_train=0
# best_SVR_MAPE_test=0
SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_tsize_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_tsize_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for ts in test_size:
    for a in gamma_nums:
        for b in C_values:
            random.seed(a*10)
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            #예측값 생성
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            SVR_RMSE_train_list.append(SVR_RMSE_train)
            SVR_RMSE_test_list.append(SVR_RMSE_test)
            SVR_MAPE_train_list.append(SVR_MAPE_train)
            SVR_MAPE_test_list.append(SVR_MAPE_test)
            
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
            SVR_par_diff_list.append(SVR_RMSE_diff)
            SVR_par_tsize_list.append(ts)
            SVR_par_gamma_list.append(a)
            SVR_par_cval_list.append(b)
            
#             if best_SVR_RMSE_diff>SVR_RMSE_diff:
#                 best_SVR_RMSE_diff = SVR_RMSE_diff
#                 best_SVR_RMSE_train = SVR_RMSE_train
#                 best_SVR_RMSE_test = SVR_RMSE_test
#                 best_SVR_MAPE_train = SVR_MAPE_train
#                 best_SVR_MAPE_test = SVR_MAPE_test
#                 SVR_C = count+1
#                 SVR_TS = ts
#                 SVR_A = a
#                 SVR_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


----progress...----
 1.00% , case:1 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 500 , difference(%) : 9.95
 ---  train RMSE : 4931 , test RMSE : 5476
---  train MAPE : 10.49 , test MAPE : 11.57
 2.00% , case:2 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 1000 , difference(%) : 1.56
 ---  train RMSE : 4426 , test RMSE : 4358
---  train MAPE : 9.18 , test MAPE : 9.53
 3.00% , case:3 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 5000 , difference(%) : 1.98
 ---  train RMSE : 2474 , test RMSE : 2524
---  train MAPE : 5.32 , test MAPE : 5.47
 4.00% , case:4 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 10000 , difference(%) : 14.01
 ---  train RMSE : 2001 , test RMSE : 2327
---  train MAPE : 4.57 , test MAPE : 5.76
 5.00% , case:5 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 50000 , difference(%) : 9.42
 ---  train RMSE : 1491 , test RMSE : 1646
---  train MAPE : 3.81 , test MAPE : 4.59
 6.00% , case:6 , test_size: 0.1 , Gamma_size: 0.001 ,  C_values: 500 ,

In [35]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = SVR_min_vars_tsize_list[i]
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
            #예측값 생성
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

    #     if SVR_Best_min>SVR_Best_MAPE_test:
    #         SVR_Best_min = SVR_Best_MAPE_test
    #         SVR_Best_TS = ts
    #         SVR_Best_A = a
    #         SVR_Best_B = b
    #         SVR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    


PARAMETERS FOR BEST SVR...
test_size : 0.15 , Gamma_size: 0.00015 ,  C_values: 500
test_size : 0.15 , Gamma_size: 0.00015 ,  C_values: 1000
test_size : 0.15 , Gamma_size: 0.00015 ,  C_values: 5000
test_size : 0.15 , Gamma_size: 0.00015 ,  C_values: 10000
test_size : 0.15 , Gamma_size: 0.00015 ,  C_values: 50000


...testing...

 1.00% , case:1 , test_size: 0.15 , Tree_num: 0.00015 , Var_num: 500 , difference(%) : 7.94
 ---  train RMSE : 5028 , test RMSE : 4658
---  train MAPE : 10.65 , test MAPE : 10.43
 2.00% , case:2 , test_size: 0.15 , Tree_num: 0.00015 , Var_num: 500 , difference(%) : 11.95
 ---  train RMSE : 5070 , test RMSE : 4529
---  train MAPE : 10.8 , test MAPE : 10.33
 3.00% , case:3 , test_size: 0.15 , Tree_num: 0.00015 , Var_num: 500 , difference(%) : 3.28
 ---  train RMSE : 4980 , test RMSE : 5149
---  train MAPE : 10.6 , test MAPE : 11.84
 4.00% , case:4 , test_size: 0.15 , Tree_num: 0.00015 , Var_num: 500 , difference(%) : 15.90
 ---  train RMSE : 5087 , test RMSE : 438

In [36]:
SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
print("SVR best 5 models")
SVR_best_total_dataframe_rbf

SVR best 5 models


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,4999.850000,4440.450000,2542.350000,2037.150000,1499.550000
RMSE test Mean,5000.000000,4484.000000,2788.550000,2215.900000,1668.500000
RMSE train Stdev,55.909100,46.993058,35.729924,22.213228,18.829432
RMSE test Stdev,427.806732,305.868599,293.330441,218.938096,148.245911
MAPE train Mean,10.628000,9.221000,5.395500,4.615000,3.793000
MAPE test Mean,11.081000,9.732500,6.109000,5.177000,4.489000
MAPE train Stdev,0.119942,0.067742,0.040679,0.048939,0.035086
MAPE test Stdev,0.778376,0.394498,0.289100,0.231778,0.219155
RMSE diff(%) Mean,7.803853,6.332650,10.163337,9.196710,10.985670
RMSE diff(%) Stdev,5.496503,4.540740,8.123668,7.275993,6.637523


### EXTRA... SVM LINEAR KERNEL

In [37]:
# gamma_nums = [0.001,0.01,0.1,1]
# C_values= [10,100,1000,10000]
# x = len(gamma_nums)
# y = len(C_values)
# count=0
# # Random Forest Relaxed Search
# count=0

# # best_SVR_RMSE_diff=100
# # best_SVR_RMSE_train=0
# # best_SVR_RMSE_test=0
# # best_SVR_MAPE_train=0
# # best_SVR_MAPE_test=0
# SVR_RMSE_train_list =[]
# SVR_RMSE_test_list =[]
# SVR_MAPE_train_list =[]
# SVR_MAPE_test_list =[]
# SVR_par_diff_list=[]
# SVR_par_tsize_list=[]
# SVR_par_gamma_list=[]
# SVR_par_cval_list=[]
# SVR_min_vars_tsize_list=[]
# SVR_min_vars_gamma_list=[]
# SVR_min_vars_cval_list=[]   

# print("----progress...----")

# for ts in test_size:
#     for a in gamma_nums:
#         for b in C_values:
#             random.seed(a*10)
#             X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#             SVR_model = svm.SVR(kernel='linear',gamma=a,C=b)
#             SVR_model.fit(X_train, Y_train)
            
#             #예측값 생성
#             SVR_prediction_test = SVR_model.predict(X_test)
#             SVR_prediction_test = SVR_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#             SVR_train_test = SVR_model.predict(X_train)
#             SVR_train_test = SVR_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#             SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
#             SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#             SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
#             SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
#             # 값들을 위에 지정한 list 에다 다 저장
#             SVR_RMSE_train_list.append(SVR_RMSE_train)
#             SVR_RMSE_test_list.append(SVR_RMSE_test)
#             SVR_MAPE_train_list.append(SVR_MAPE_train)
#             SVR_MAPE_test_list.append(SVR_MAPE_test)
            
#             SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
#             SVR_par_diff_list.append(SVR_RMSE_diff)
#             SVR_par_tsize_list.append(ts)
#             SVR_par_gamma_list.append(a)
#             SVR_par_cval_list.append(b)
            
# #             if best_SVR_RMSE_diff>SVR_RMSE_diff:
# #                 best_SVR_RMSE_diff = SVR_RMSE_diff
# #                 best_SVR_RMSE_train = SVR_RMSE_train
# #                 best_SVR_RMSE_test = SVR_RMSE_test
# #                 best_SVR_MAPE_train = SVR_MAPE_train
# #                 best_SVR_MAPE_test = SVR_MAPE_test
# #                 SVR_C = count+1
# #                 SVR_TS = ts
# #                 SVR_A = a
# #                 SVR_B = b
         
                
#             count+=1
#             progress = round(count/(x*y*ts_len)*100,2)
            
#             print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# # print("\n\n")
# # print("Best Case")
# # print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
# #       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
# #       "---  train MAPE : {} , test MAPE : {}"
# #       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


# for i in range(5):
#     val = min(SVR_par_diff_list)
#     index = SVR_par_diff_list.index(val)
#     SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
#     SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
#     SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


In [38]:
# print("PARAMETERS FOR BEST SVR...")
# for i in range(5):
#     print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

# print("\n\n...testing...\n")
# test_times=5
# progress=0
# count=0
# SVR_Best_RMSE_train_list=[]
# SVR_Best_RMSE_test_list=[]
# SVR_Best_MAPE_train_list=[]
# SVR_Best_MAPE_test_list=[]
# SVR_Best_RMSE_diff_list=[]

# SVR_Best_total_data=[]

# for i in range(5):
#     temp_SVR_Best_RMSE_train_list=[]
#     temp_SVR_Best_RMSE_test_list=[]
#     temp_SVR_Best_MAPE_train_list=[]
#     temp_SVR_Best_MAPE_test_list=[]
#     temp_SVR_Best_RMSE_diff_list=[]
#     for k in range(test_times):
#         ts = SVR_min_vars_tsize_list[i]
#         a = SVR_min_vars_gamma_list[i]
#         b = SVR_min_vars_cval_list[i]
#         random.seed((k+1)*10)

#         #훈련셋, 시험셋 분류 
#         X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#         SVR_Best_model = svm.SVR(kernel='linear',gamma=a,C=b)
#         SVR_Best_model.fit(X_train, Y_train)
            
#             #예측값 생성
#         SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
#         SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#         SVR_Best_train_test = SVR_Best_model.predict(X_train)
#         SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#         SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
#         SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#         SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
#         SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
#         SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
#         # 값들을 위에 지정한 list 에다 다 저장
#         temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
#         temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
#         temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
#         temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
#         temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

#     #     if SVR_Best_min>SVR_Best_MAPE_test:
#     #         SVR_Best_min = SVR_Best_MAPE_test
#     #         SVR_Best_TS = ts
#     #         SVR_Best_A = a
#     #         SVR_Best_B = b
#     #         SVR_Best_C = count+1

#         count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))


In [39]:
# SVR1=[]
# SVR2=[]
# SVR3=[]
# SVR4=[]
# SVR5=[]

# for i in range(50):
#     k=i//10
#     if k==0:
#         SVR1.append(SVR_Best_total_data[i])
#     if k==1:
#         SVR2.append(SVR_Best_total_data[i])
#     if k==2:
#         SVR3.append(SVR_Best_total_data[i])
#     if k==3:
#         SVR4.append(SVR_Best_total_data[i])
#     if k==4:
#         SVR5.append(SVR_Best_total_data[i])

    
# SVR_s1 = pd.Series(SVR1,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 1')

# SVR_s2 = pd.Series(SVR2,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 2')
# SVR_s3 = pd.Series(SVR3,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 3')
# SVR_s4 = pd.Series(SVR4,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 4')
# SVR_s5 = pd.Series(SVR5,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 5')

# SVR_best_total_dataframe_linear = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
# print("SVR best 5 models")
# SVR_best_total_dataframe_linear

### 3. Linear Regression ... <br>
what parameters are there here?

In [40]:
times=10
count=0

# best_LR_RMSE_diff=100
# best_LR_RMSE_train=0
# best_LR_RMSE_test=0
# best_LR_MAPE_train=0
# best_LR_MAPE_test=0
LR_RMSE_train_list =[]
LR_RMSE_test_list =[]
LR_MAPE_train_list =[]
LR_MAPE_test_list =[]
LR_par_diff_list=[]
LR_par_tsize_list=[]
LR_min_vars_tsize_list=[] 

print("----progress...----")

for ts in test_size:
    for i in range(times):
        random.seed(a*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_model=LinearRegression()
        LR_model.fit(X_train, Y_train)
        LR_prediction_test = LR_model.predict(X_test)
        LR_prediction_test = LR_prediction_test.astype('int')
        LR_train_test = LR_model.predict(X_train)
        LR_train_test = LR_train_test.astype('int')
    
        #훈련셋 RMSE & MAPE
        LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
        LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
            
        #시험셋 RMSE & MAPE
        LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')
        LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)
        
        # 값들을 위에 지정한 list 에다 다 저장
        LR_RMSE_train_list.append(LR_RMSE_train)
        LR_RMSE_test_list.append(LR_RMSE_test)
        LR_MAPE_train_list.append(LR_MAPE_train)
        LR_MAPE_test_list.append(LR_MAPE_test)
            
        LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
            
        LR_par_diff_list.append(LR_RMSE_diff)
        LR_par_tsize_list.append(ts)
            
#             if best_LR_RMSE_diff>LR_RMSE_diff:
#                 best_LR_RMSE_diff = LR_RMSE_diff
#                 best_LR_RMSE_train = LR_RMSE_train
#                 best_LR_RMSE_test = LR_RMSE_test
#                 best_LR_MAPE_train = LR_MAPE_train
#                 best_LR_MAPE_test = LR_MAPE_test
#                 LR_C = count+1
#                 LR_TS = ts
#                 LR_A = a
#                 LR_B = b
         
                
        count+=1
        progress = round(count/(ts_len*times)*100,2)
            
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
        "---  train RMSE : {} , test RMSE : {}\n"
        "---  train MAPE : {} , test MAPE : {}"
                .format(progress,count,ts,LR_RMSE_diff,LR_RMSE_train,LR_RMSE_test,LR_MAPE_train,LR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(LR_C,LR_TS,best_LR_RMSE_train,best_LR_RMSE_test,best_LR_RMSE_diff,best_LR_MAPE_train,best_LR_MAPE_test))
                


for i in range(5):
    val = min(LR_par_diff_list)
    index = LR_par_diff_list.index(val)
    LR_min_vars_tsize_list.append(LR_par_tsize_list.pop(index))


----progress...----
 2.00% , case:1 , test_size: 0.1 , difference(%) : 7.21
 ---  train RMSE : 3449 , test RMSE : 3217
---  train MAPE : 12.05 , test MAPE : 11.58
 4.00% , case:2 , test_size: 0.1 , difference(%) : 4.43
 ---  train RMSE : 3411 , test RMSE : 3569
---  train MAPE : 11.93 , test MAPE : 13.05
 6.00% , case:3 , test_size: 0.1 , difference(%) : 10.95
 ---  train RMSE : 3384 , test RMSE : 3800
---  train MAPE : 11.93 , test MAPE : 12.05
 8.00% , case:4 , test_size: 0.1 , difference(%) : 0.06
 ---  train RMSE : 3427 , test RMSE : 3429
---  train MAPE : 12.01 , test MAPE : 11.83
 10.00% , case:5 , test_size: 0.1 , difference(%) : 0.20
 ---  train RMSE : 3427 , test RMSE : 3420
---  train MAPE : 11.98 , test MAPE : 12.12
 12.00% , case:6 , test_size: 0.1 , difference(%) : 0.12
 ---  train RMSE : 3427 , test RMSE : 3423
---  train MAPE : 12.04 , test MAPE : 11.9
 14.00% , case:7 , test_size: 0.1 , difference(%) : 5.04
 ---  train RMSE : 3408 , test RMSE : 3589
---  train MAPE : 11

In [41]:
print("PARAMETERS FOR BEST LR...")
for i in range(5):
    print("test_size : {} ".format(LR_min_vars_tsize_list[i]))

print("...testing...")

progress=0
count=0
LR_Best_RMSE_train_list=[]
LR_Best_RMSE_test_list=[]
LR_Best_MAPE_train_list=[]
LR_Best_MAPE_test_list=[]
LR_Best_RMSE_diff_list=[]

LR_Best_total_data=[]

for i in range(5):
    temp_LR_Best_RMSE_train_list=[]
    temp_LR_Best_RMSE_test_list=[]
    temp_LR_Best_MAPE_train_list=[]
    temp_LR_Best_MAPE_test_list=[]
    temp_LR_Best_RMSE_diff_list=[]
    
    for k in range(test_times):
        ts = LR_min_vars_tsize_list[i]
        random.seed((k+1)*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_Best_model=LinearRegression()
        LR_Best_model.fit(X_train, Y_train)
        LR_Best_prediction_test = LR_Best_model.predict(X_test)
        LR_Best_prediction_test = LR_Best_prediction_test.astype('int')
        LR_Best_train_test = LR_Best_model.predict(X_train)
        LR_Best_train_test = LR_Best_train_test.astype('int')

        #훈련셋 RMSE & MAPE
        LR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_Best_train_test)).astype('int')
        LR_Best_MAPE_train = round(np.mean(np.abs((Y_train - LR_Best_train_test) / Y_train)) * 100 ,2)

        #시험셋 RMSE & MAPE
        LR_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, LR_Best_prediction_test)).astype('int')
        LR_Best_MAPE_test = round(np.mean(np.abs((Y_test - LR_Best_prediction_test) / Y_test)) * 100 ,2)

        LR_Best_RMSE_diff = np.abs((LR_Best_RMSE_test-LR_Best_RMSE_train)/LR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_LR_Best_RMSE_train_list.append(LR_Best_RMSE_train)
        temp_LR_Best_RMSE_test_list.append(LR_Best_RMSE_test)
        temp_LR_Best_MAPE_train_list.append(LR_Best_MAPE_train)
        temp_LR_Best_MAPE_test_list.append(LR_Best_MAPE_test)
        temp_LR_Best_RMSE_diff_list.append(LR_Best_RMSE_diff)
            

    #     if LR_Best_min>LR_Best_MAPE_test:
    #         LR_Best_min = LR_Best_MAPE_test
    #         LR_Best_TS = ts
    #         LR_Best_A = a
    #         LR_Best_B = b
    #         LR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,LR_Best_RMSE_diff,LR_Best_RMSE_train,LR_Best_RMSE_test,LR_Best_MAPE_train,LR_Best_MAPE_test))
    
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_diff_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_diff_list))
    

#     LR_Best_RMSE_train_list.append(temp_LR_Best_RMSE_train_list)
#     LR_Best_RMSE_test_list.append(temp_LR_Best_RMSE_test_list)
#     LR_Best_MAPE_train_list.append(temp_LR_Best_MAPE_train_list)
#     LR_Best_MAPE_test_list.append(temp_LR_Best_MAPE_test_list)
#     LR_Best_RMSE_diff_list.append(temp_LR_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(LR_Best_C,LR_Best_TS,LR_Best_A,LR_Best_B,LR_Best_min))



PARAMETERS FOR BEST LR...
test_size : 0.2 
test_size : 0.2 
test_size : 0.2 
test_size : 0.2 
test_size : 0.2 
...testing...
 1.00% , case:1 , test_size: 0.2 , difference(%) : 1.36
 ---  train RMSE : 3440 , test RMSE : 3394
---  train MAPE : 11.93 , test MAPE : 12.49
 2.00% , case:2 , test_size: 0.2 , difference(%) : 3.52
 ---  train RMSE : 3403 , test RMSE : 3527
---  train MAPE : 11.89 , test MAPE : 12.59
 3.00% , case:3 , test_size: 0.2 , difference(%) : 4.00
 ---  train RMSE : 3456 , test RMSE : 3323
---  train MAPE : 12.08 , test MAPE : 12.11
 4.00% , case:4 , test_size: 0.2 , difference(%) : 6.09
 ---  train RMSE : 3469 , test RMSE : 3270
---  train MAPE : 12.14 , test MAPE : 11.94
 5.00% , case:5 , test_size: 0.2 , difference(%) : 1.54
 ---  train RMSE : 3437 , test RMSE : 3385
---  train MAPE : 12.07 , test MAPE : 11.77
 6.00% , case:6 , test_size: 0.2 , difference(%) : 3.45
 ---  train RMSE : 3450 , test RMSE : 3335
---  train MAPE : 12.09 , test MAPE : 11.7
 7.00% , case:7 , 

In [42]:


LR1=[]
LR2=[]
LR3=[]
LR4=[]
LR5=[]

for i in range(50):
    k=i//10
    if k==0:
        LR1.append(LR_Best_total_data[i])
    if k==1:
        LR2.append(LR_Best_total_data[i])
    if k==2:
        LR3.append(LR_Best_total_data[i])
    if k==3:
        LR4.append(LR_Best_total_data[i])
    if k==4:
        LR5.append(LR_Best_total_data[i])
    
LR_s1 = pd.Series(LR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

LR_s2 = pd.Series(LR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
LR_s3 = pd.Series(LR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
LR_s4 = pd.Series(LR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
LR_s5 = pd.Series(LR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

LR_best_total_dataframe = pd.concat([LR_s1,LR_s2,LR_s3,LR_s4,LR_s5],axis=1)

LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3427.050000,3422.750000,3430.400000,3431.050000,3423.250000
RMSE test Mean,3431.900000,3447.000000,3418.000000,3412.850000,3447.200000
RMSE train Stdev,27.273568,25.238611,30.952221,25.421398,21.967874
RMSE test Stdev,108.058271,102.474387,122.742413,102.103024,88.634982
MAPE train Mean,12.005000,11.979500,11.995000,12.003500,11.987000
MAPE test Mean,12.055500,12.128000,12.059000,11.910000,11.985000
MAPE train Stdev,0.095420,0.093246,0.110521,0.086271,0.089783
MAPE test Stdev,0.361725,0.377738,0.373844,0.357071,0.229946
RMSE diff(%) Mean,3.377087,2.997363,3.221503,3.078587,2.795495
RMSE diff(%) Stdev,2.047189,2.156841,3.132086,2.268900,1.657133


In [15]:
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,5901.000000,3080.200000,1934.200000,1375.800000,5836.400000
RMSE test Mean,5789.300000,3140.900000,2052.900000,1567.100000,5940.800000
RMSE train Stdev,90.743595,103.052220,36.790760,26.656331,64.417699
RMSE test Stdev,90.743595,103.052220,36.790760,26.656331,64.417699
MAPE train Mean,20.080000,10.990000,6.770000,4.572000,19.839000
MAPE test Mean,19.912000,11.094000,7.117000,5.005000,19.847000
MAPE train Stdev,0.452482,0.395929,0.144291,0.088747,0.176887
MAPE test Stdev,0.452482,0.395929,0.144291,0.088747,0.176887
RMSE diff(%) Mean,4.380071,4.623703,6.289273,11.936670,5.320058
RMSE diff(%) Stdev,2.896110,2.752326,3.630406,5.717231,3.325602


In [16]:
SVR_best_total_dataframe_rbf

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,2081.600000,1978.000000,2686.300000,1430.200000,2126.300000
RMSE test Mean,2189.000000,2188.000000,2720.700000,1760.600000,2174.800000
RMSE train Stdev,39.441602,25.262621,43.660165,42.300827,20.966879
RMSE test Stdev,39.441602,25.262621,43.660165,42.300827,20.966879
MAPE train Mean,4.660000,4.501000,5.588000,3.636000,4.687000
MAPE test Mean,5.207000,5.177000,6.106000,4.596000,5.221000
MAPE train Stdev,0.038987,0.061392,0.067941,0.050239,0.052163
MAPE test Stdev,0.038987,0.061392,0.067941,0.050239,0.052163
RMSE diff(%) Mean,9.410394,9.154732,7.936839,17.593459,7.913053
RMSE diff(%) Stdev,5.801017,6.132544,3.108907,10.621967,6.660207


In [17]:
LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3422.000000,3429.600000,3414.200000,3427.200000,3430.800000
RMSE test Mean,3454.800000,3411.200000,3502.500000,3427.400000,3417.800000
RMSE train Stdev,27.254357,27.408758,19.301813,29.219172,25.443270
RMSE test Stdev,27.254357,27.408758,19.301813,29.219172,25.443270
MAPE train Mean,11.960000,12.000000,11.962000,11.986000,12.013000
MAPE test Mean,12.055000,11.768000,12.197000,12.058000,11.930000
MAPE train Stdev,0.092952,0.078230,0.088068,0.091345,0.104312
MAPE test Stdev,0.092952,0.078230,0.088068,0.091345,0.104312
RMSE diff(%) Mean,4.427254,4.776021,3.596998,3.598775,3.194342
RMSE diff(%) Stdev,2.983396,2.484808,2.436418,2.372767,1.820745
